In [5]:
import tensorflow as tf
import numpy as np
import os

# TensorBoard

Tensorboard is a monitoring tools for training different models. It captures values during training so user can monitor the training process and figure out potential problem of a structure or training strategy.
To visualise the tensorboard in jupyter notebook, we can strip the graph structure from the graph and display it in a HTML frame:(code copied from stackOverFlow)

# Plot within jupyter

In [ ]:
#To show the tensorBoard
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
#Create variable
var3=tf.get_variable('Var3',shape=(1,1),dtype=tf.float64)

#The variable will be created with the value used as its default value as it is initialised
init = tf.Variable(451, tf.int16)
const1=tf.constant(1,dtype=tf.float64)

#Operations
add1=var3+const1
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
tf.reset_default_graph()

# Variables Logging

To create a tensorboard, we will need to create a directory to store the logs. 

In [11]:
#Opening new directory
from datetime import datetime
now = datetime.now()
logdir = 'C:/Users/kin.poon/Desktop/tf_logs' + now.strftime("%Y%m%d-%H%M%S") + "/"
os.mkdir(logdir)
print('TensorBoard CMD:tensorboard --logdir='+logdir)

TensorBoard CMD:tensorboard --logdir=C:/Users/kin.poon/Desktop/tf_logs20180718-151516/


## Model

When the model is defined, we can define a summary object, in which we can record different variables in the graph. This summary objects need to be executed in the session together with the calculations to take the snapshots. There are different type of summary we can get. These summaries can help tracking the changes of variables so the user can observe the changes and debug the structure/procedure accordingly. For example, being able to observe the activation values will help getting insight on whether the information is passing through the network efficiently. The practitioner may then adjust the initiallisation, apply batch normalisation or re-design the structure to improve the performance.

In [ ]:

#-----
#Cost and weights defined
#-----

tf.summary.scalar('Cost',cost) #Recording scalar 'Cost'.
tf.summary.histogram('Weights',encoded) #Recording tensor 'Weights' and plot it as histogram.
merged=tf.summary.merge_all()
saver = tf.train.Saver()

After creating the logs of different variables, we can merge all the summary into a single summary object for collective execution.

## Training iteration

In [ ]:
#-----
#Session defined
#-----

#Define the writer with the graph, which contains the summaries and merged summary
writer = tf.summary.FileWriter(logdir, sess.graph)

# Training epochs
for e in range(epochs):
    for ii in range(num_examples//batch_size):
        #Running the variable merged in the session to collect the merged summary 
        summary, batch_cost, _ = sess.run([merged,cost, opt], feed_dict={inputs_: imgs, targets_: imgs})
        print("Running epochs {}, batch {}".format(e+1,ii+1))
        #Writing the summary down to the writer, which is the returned result of the collected summary.
        writer.add_summary(summary, ii)



After executing the summary, we will need to create a summary writer to write down the log for tensorboard to display.

# Embeddings

Embeddings is a common technique used for NLP. It allows pratitioners to embedding data objects into a latent space which retain only significant features of the data(if the embedding is successful). Being able to visualise embeddings is important to determine whether the embedding training is successful or not.

In [14]:
from tensorflow.contrib.tensorboard.plugins import projector

# ----------------- Logging the embeded tensor ---------------------------#
# Create randomly initialized embedding weights which will be trained.
N = 2000 # Number of items (vocab size).
D = 200 # Dimensionality of the embedding.
embedding_var = tf.Variable(tf.random_normal([N,D]), name='word_embedding')
# ------------------------------------------------------------------------#


#----------------- Configuring the projector --------------------------------------#
# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
# Create the config object for the projector
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.

# Add an embedding object in the config
embedding = config.embeddings.add()
# Linking the embedding object
embedding.tensor_name = embedding_var.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join('C:/Users/kin.poon/Desktop/tf_meta_test/', 'metaData.tsv')
#----------------------------------------------------------------------------------#


#----------------- For adding image thumbnails ------------------------------------#
'''
embedding.sprite.image_path = PATH_TO_SPRITE_IMAGE
# Specify the width and height of a single thumbnail.
embedding.sprite.single_image_dim.extend([w, h])
'''
#----------------------------------------------------------------------------------#


#----------------- Writing the embeddings down ------------------------------------#
# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(logdir)

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)
#----------------------------------------------------------------------------------#

In [19]:

with tf.Session() as sess:
    saver = tf.train.Saver([embedding_var])
    sess.run(tf.global_variables_initializer())
    saver.save(sess, os.path.join(logdir, 'embeddings.ckpt'))